<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import f1_score
import lightgbm as lgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm.notebook import tqdm
import spacy


In [2]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')
df.head()


,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [3]:

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nlp = spacy.load('en_core_web_sm')


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
nlp = spacy.load('en_core_web_sm')

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()

    # Remove punctuation and numbers
    text = re.sub(r'[^a-z\s]', '', text)

    # Tokenize using spaCy
    doc = nlp(text)
    words = [token.lemma_ for token in doc if not token.is_stop]

    return ' '.join(words)


In [5]:
sentence = "The striped bats are hanging on their feet for best"
preprocess_text(sentence)

'stripe bat hang foot good'

In [6]:
# Apply the preprocessing to the 'text' column
tqdm.pandas()
df['processed_text'] = df['text'].progress_apply(preprocess_text)

df[['text', 'processed_text']].head()


  0%|          | 0/159292 [00:00<?, ?it/s]

,text,processed_text
0,Explanation\nWhy the edits made under my usern...,explanation \n edit username hardcore metallic...
1,D'aww! He matches this background colour I'm s...,daww match background colour m seemingly stuck...
2,"Hey man, I'm really not trying to edit war. It...",hey man m try edit war guy constantly remove r...
3,"""\nMore\nI can't make any real suggestions on ...",\n \n not real suggestion improvement wonder...
4,"You, sir, are my hero. Any chance you remember...",sir hero chance remember page s


In [7]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['processed_text'], df['toxic'], test_size=0.20, random_state=42)


## Обучение

In [8]:

# Define the model pipeline with a placeholder for the regularization parameter C
model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(solver='liblinear')),
])

# Define the parameters for grid search
# Note: C = 1/alpha, so we'll take reciprocals
# https://stackoverflow.com/questions/67513075/what-is-c-parameter-in-sklearn-logistic-regression
parameters = {
    'clf__C': [1/0.001, 1/0.01, 1/0.1, 1/1.0, 1/10.0],
}

# Perform grid search
grid_search = GridSearchCV(model, parameters, cv=2, scoring='f1', verbose=10)
grid_search.fit(X_train, y_train)

best_logreg = grid_search.best_estimator_
grid_search.best_params_, grid_search.best_score_

Fitting 2 folds for each of 5 candidates, totalling 10 fits
[CV 1/2; 1/5] START clf__C=1000.0...............................................
[CV 1/2; 1/5] END .............................clf__C=1000.0; total time=  29.0s
[CV 2/2; 1/5] START clf__C=1000.0...............................................
[CV 2/2; 1/5] END .............................clf__C=1000.0; total time=  26.7s
[CV 1/2; 2/5] START clf__C=100.0................................................
[CV 1/2; 2/5] END ..............................clf__C=100.0; total time=  20.0s
[CV 2/2; 2/5] START clf__C=100.0................................................
[CV 2/2; 2/5] END ..............................clf__C=100.0; total time=  26.5s
[CV 1/2; 3/5] START clf__C=10.0.................................................
[CV 1/2; 3/5] END ...............................clf__C=10.0; total time=  12.9s
[CV 2/2; 3/5] START clf__C=10.0.................................................
[CV 2/2; 3/5] END ...............................

({'clf__C': 10.0}, 0.7651707600369475)

In [9]:
grid_search.best_params_, grid_search.best_score_

({'clf__C': 10.0}, 0.7651707600369475)

In [10]:
# Create a GradientBoostingClassifier instance
model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('boosting', lgb.LGBMClassifier())
])

# Define the parameter grid for GridSearchCV
parameters = {
    'tfidf__max_features': [10_000],
    'boosting__n_estimators': [30],
    'boosting__learning_rate': [0.10, 0.50],
    'boosting__max_depth': [30, 60],
    'boosting__num_leaves': [30, 60],
    'boosting__random_state': [42],
    'boosting__num_threads': [4],
}

# Create GridSearchCV instance
grid_search = GridSearchCV(model, parameters, cv=2, scoring='f1', verbose=10)

# Train the model with GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best parameters and score
best_boost = grid_search.best_estimator_
grid_search.best_params_, grid_search.best_score_

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV 1/2; 1/8] START boosting__learning_rate=0.1, boosting__max_depth=30, boosting__n_estimators=30, boosting__num_leaves=30, boosting__num_threads=4, boosting__random_state=42, tfidf__max_features=10000
[LightGBM] [Warning] num_threads is set=4, n_jobs=-1 will be ignored. Current value: num_threads=4
[CV 1/2; 1/8] END boosting__learning_rate=0.1, boosting__max_depth=30, boosting__n_estimators=30, boosting__num_leaves=30, boosting__num_threads=4, boosting__random_state=42, tfidf__max_features=10000; total time=  27.6s
[CV 2/2; 1/8] START boosting__learning_rate=0.1, boosting__max_depth=30, boosting__n_estimators=30, boosting__num_leaves=30, boosting__num_threads=4, boosting__random_state=42, tfidf__max_features=10000
[LightGBM] [Warning] num_threads is set=4, n_jobs=-1 will be ignored. Current value: num_threads=4
[CV 2/2; 1/8] END boosting__learning_rate=0.1, boosting__max_depth=30, boosting__n_estimators=30, boosting__num_lea

({'boosting__learning_rate': 0.5,
  'boosting__max_depth': 60,
  'boosting__n_estimators': 30,
  'boosting__num_leaves': 60,
  'boosting__num_threads': 4,
  'boosting__random_state': 42,
  'tfidf__max_features': 10000},
 0.7572480326668095)

In [11]:
grid_search.best_params_, grid_search.best_score_

({'boosting__learning_rate': 0.5,
  'boosting__max_depth': 60,
  'boosting__n_estimators': 30,
  'boosting__num_leaves': 60,
  'boosting__num_threads': 4,
  'boosting__random_state': 42,
  'tfidf__max_features': 10000},
 0.7572480326668095)

In [12]:
best_model = best_logreg

In [13]:
y_pred = best_model.predict(X_test)
f1 = f1_score(y_test, y_pred)
f1

0.7810423224903813

## Выводы

Логистическая регрессия на TFIDF сработала достаточно хорошо - удовлетворяет требованиям заказчика.